In [1]:
#default_exp hypster_prepare

In [2]:
#export
from hypster.oo_hp import *

In [3]:
#export
from inspect import signature
import functools
from collections import OrderedDict

In [4]:
#export
def set_names(arg_names, args, kwargs):
    for i, arg in enumerate(args):
        if isinstance(arg, HpExpression) and arg.manual_name == False:
            #TODO: what if it only contains HpExpression?
            #TODO: handle *args
            arg.name = arg_names[i]
    for key, value in kwargs.items():
        if isinstance(value, HpExpression) and value.manual_name == False: #TODO: what if it only contains HpExpression?
            value.name = key

In [5]:
#export
class HypsterPrepare(HypsterBase):
    def __init__(self, call, base_call, *args, **kwargs):
        #allow future option to add a prefix of a model name
        self.call            = call
        self.base_call       = base_call
        self.args            = args
        self.kwargs          = kwargs
        self.trials_sampled  = set()
        self.studies_sampled = set()
        self.base_object     = None
        self.result          = None
        
        # if argument is an iterable -> (start_mom, ..., ...) use its object name
        # Normalize(mean=HpFloat(0.001, 10.4)) --> change from "mean" to normalize_mean if it's not the main prepare?
        # procs = [Categorify, imp, HpToggle(norm)] --> change toggle_hypsterprepare to toggle_normalize
        # fit_method = learner.fit_one_cycle(2, lr) --> during sampling, get the function signature and change the name
        
        if callable(call):
            self.arg_names = list(signature(call).parameters.keys())
            set_names(self.arg_names, self.args, self.kwargs)
        
    def sample(self, trial):
        if trial.study.study_name not in self.studies_sampled:
            self.trials_sampled = set()
        elif trial.number in self.trials_sampled:
            return self.result           
        
        if self.base_call is not None:
            self.base_object = self.base_call.sample(trial)

        self.sampled_args   = populate_iterable(self.args, trial)
        self.sampled_kwargs = populate_dict(self.kwargs, trial)
        
        self.trials_sampled.add(trial.number)
        self.studies_sampled.add(trial.study.study_name)

        if self.base_object:
            if len(self.sampled_args) == 0 and len(self.sampled_kwargs) == 0:
                self.result = getattr(self.base_object, self.call)
            else:
                self.result = getattr(self.base_object, self.call)(*self.sampled_args, **self.sampled_kwargs)
        else:
            self.result = self.call(*self.sampled_args, **self.sampled_kwargs)
        return self.result
    
    def __call__(self, *args, **kwargs):
        #print(f"args {args}, kwargs {kwargs}")
        self.args = args
        self.kwargs = kwargs
        return self
    
    def __getattr__(self, name, *args, **kwargs):
        #print(f"name {name}, args {args}, kwargs {kwargs}")
        return HypsterPrepare(name, self, *args, **kwargs)

In [6]:
#export
def prepare(call):
    @functools.wraps(call)
    def wrapper_decorator(*args, **kwargs):
        all_args = list(args) + list(kwargs.values())
        if any([contains_hypster(arg, HYPSTER_TYPES) for arg in all_args]):
            return HypsterPrepare(call, None, *args, **kwargs)
        else:
            return call(*args, **kwargs)
    return wrapper_decorator

# Test Naming

In [7]:
fill_strategy = HpCategorical(["mode", "median"])

In [8]:
class FillMissing:
    @auto_assign
    def __init__(self, fill_strategy="mode", add_col=False): pass

In [9]:
FillMissing = prepare(FillMissing)

In [10]:
imp = FillMissing(fill_strategy=fill_strategy, add_col=False)

In [11]:
signature(FillMissing).parameters

mappingproxy({'fill_strategy': <Parameter "fill_strategy='mode'">,
              'add_col': <Parameter "add_col=False">})

In [12]:
imp.kwargs["fill_strategy"].name

'fill_strategy'

# Test Prepare

In [13]:
def foo(a, b, c="he", d=None, **kwargs):
    if c is not None:
        print(f"{a} and {b} and {c}")
    else:
        print(f"{a} and {b}")
        
    return f"returned {a} and {b} and {c}"    

In [14]:
foo("hi", "helol", c="errr")

hi and helol and errr


'returned hi and helol and errr'

In [15]:
foo = prepare(foo)

In [16]:
a = foo("helo", HpCategorical(["go", "no"]))

In [17]:
foo("hi", HpCategorical(["hello", "hola!"]))

In [18]:
def foo2(a, b="hi!", c=None, **kwargs):
    if c is not None:
        print(f"{a} and {b} and {c}")
    else:
        print(f"{a} and {b}")
    
    return f"returned {a} and {b} and {c}"

In [19]:
class Cls:
    def __init__(self, a, b="hi!", c=None, **kwargs):
        if c is not None:
            print(f"{a} and {b} and {c}")
        else:
            print(f"{a} and {b}")

        #return f"returned {a} and {b} and {c}"
    
    def shmul(self, batch):
        return batch

In [20]:
Cls = prepare(Cls)

In [21]:
foo = prepare(foo)

In [22]:
hps = foo("hi", b="shmuli")

hi and shmuli and he


In [23]:
hps = foo("hi", b=HpBool())

In [24]:
hps = Cls("hi", b=HpCategorical(["Shmuli", "Buli"]))

In [25]:
hps.arg_names

['a', 'b', 'c', 'kwargs']

In [26]:
z = hps.shmul(batch=HpInt(1, 10))

In [27]:
#export
import optuna

In [28]:
#export
def run_func_test(x, n_trials=5):
    def objective(trial):
        print(x.sample(trial))
        return 1.0

    optuna.logging.set_verbosity(0)
    pruner = optuna.pruners.NopPruner()
    study = optuna.create_study(direction="maximize", pruner=pruner)
    study.optimize(objective, n_trials=n_trials, timeout=600)

In [29]:
run_func_test(z)

hi and Shmuli
6
hi and Buli
1
hi and Buli
2
hi and Buli
5
hi and Buli
2


In [30]:
class Cls():
    def __init__(self, name, last_name="", nickname=""):
        self.name = name
        self.last_name = last_name
        self.nickname = nickname
        print(f"{self.name} {self.last_name} {self.nickname}")  

In [31]:
c = Cls("Gilad", nickname="The King!")

Gilad  The King!


In [32]:
Cls2 = prepare(Cls)

In [33]:
x = Cls2("Gilad", HpCategorical(["The King!", "The Best King!"]))

In [34]:
run_func_test(x)

Gilad The Best King! 
Gilad The Best King! 
Gilad The King! 
Gilad The Best King! 
Gilad The Best King! 


## HpToggle

In [35]:
Cls2 = prepare(Cls)

In [36]:
x = Cls2("Gilad", last_name=HpToggle(HpCategorical(["The King!", "The Best King!"])))

In [37]:
run_func_test(x)

Gilad  
Gilad The King! 
Gilad The King! 
Gilad  
Gilad The King! 


In [38]:
from nbdev.export import notebook2script

In [39]:
notebook2script()

Converted 00_core.ipynb.
Converted 01_api.ipynb.
Converted 02_oo_hp.ipynb.
Converted 03_hypster_prepare.ipynb.
Converted 04_tabular_api.ipynb.
Converted 05_sklearn.ipynb.
Converted fastai_adult_tutorial.ipynb.
Converted index.ipynb.
